In [ ]:
# Welcome, this is our initial Random Forest model along with our resulting model after nested cross-validation
# Additionally, this includes a prediction of our Kon and Koff values for lanmodulin using the resulting model
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split 

In [ ]:
# This was our initial Kon and Koff Prediction:
pd.options.display.max_rows = 201 #increase max row display
# Read the Koff and Kon Dataset
koff = pd.read_csv("Dataset_S2.csv") # dataset
kon = pd.read_csv("Dataset_S1.csv") # dataset
# Remove all empty or NA cells
koff.dropna()
kon.dropna()

xf = koff.iloc[:,2:] # X Koff data
yf = koff.iloc[:,1] # Label Koff data
xn = kon.iloc[:,2:] # X Kon data
yn = kon.iloc[:,1] # Label Kon data

In [ ]:
from sklearn.model_selection import train_test_split 
#split into training and test data, 1/9 split
xf_train, xf_test, yf_train, yf_test = train_test_split(xf, yf, test_size = 1/9, random_state = 4)

xn_train, xn_test, yn_train, yn_test = train_test_split(xn, yn, test_size = 1/9, random_state = 4)

print(xf_test, xn_test)

       DASA  RES_P  RES_NP  RES_C  ...  S_GP_ALL2  S_GP_INT2  S_WLC_ALL2  S_WLC_INT2
31  1246.17  38.46   43.59  17.95  ...   0.000000   0.000000    0.528549         0.0
43  2030.40  29.17   47.92  22.92  ...   0.000000   0.000000   -3.608049         0.0
4   4755.11  50.47   35.51  14.02  ...   4.820282   0.000000    0.000000         0.0
15  1520.56  30.53   45.26  24.21  ...   8.302018   2.484907    5.398221         0.0
28  2633.38  57.14   22.22  20.63  ...   0.000000   0.000000    0.000000         0.0

[5 rows x 201 columns]        DASA  RES_P  RES_NP  RES_C  ...  S_GP_ALL2  S_GP_INT2  S_WLC_ALL2  S_WLC_INT2
31  1246.17  38.46   43.59  17.95  ...   0.000000   0.000000    0.528549         0.0
43  2030.40  29.17   47.92  22.92  ...   0.000000   0.000000   -3.608049         0.0
4   4755.11  50.47   35.51  14.02  ...   4.820282   0.000000    0.000000         0.0
15  1520.56  30.53   45.26  24.21  ...   8.302018   2.484907    5.398221         0.0
28  2633.38  57.14   22.22  20.63  ...   

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Fit Koff Regressor Data with 100 trees 
regressorf = RandomForestRegressor(n_estimators = 100, random_state = 0)

regressorf.fit(xf_train, yf_train)

# Fit Kon Regressor Data with 100 trees 
regressorn = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressorn.fit(xn_train, yn_train)

#Predicting test dataset
yf_pred = regressorf.predict(xf_test)
yn_pred = regressorn.predict(xn_test)

print(yf_pred)
print(yf_test)
print(yn_pred)
print(yn_test)

[-3.93851311 -2.82892082 -3.17830389 -3.25735374 -2.35208683]
31   -3.443697
43   -2.414539
4    -2.920819
15   -3.443697
28   -2.289037
Name: log10_KOFF, dtype: float64
[5.42287215 5.20849403 4.70561481 5.14405345 5.34671611]
31    5.557507
43    5.997823
4     5.544068
15    4.822822
28    6.143015
Name: log10_KON, dtype: float64


In [ ]:
from sklearn.metrics import mean_squared_error 

#Evaluate accuracy based off of mean square error
print(mean_squared_error(yf_test, yf_pred))
print(mean_squared_error(yn_test, yn_pred))

0.1043105671301628
0.41629041684426005


In [ ]:
# Using just the DASA to create a Random Forest Model, our resulting Model
# Remove all labels except for DASA 
for label in xf.columns: 
  if label != "DASA":
      xf_train = xf_train.drop(labels=label, axis=1)
      xn_train = xn_train.drop(labels=label, axis=1)
      xf_test = xf_test.drop(labels=label, axis=1)
      xn_test = xn_test.drop(labels=label, axis=1)

# Creating Koff Regressors
regressorf = RandomForestRegressor(n_estimators = 100, random_state = 0)
# Fitting Regressor
regressorf.fit(xf_train, yf_train)

# Creating Koff Regressors
regressorn = RandomForestRegressor(n_estimators = 100, random_state = 0)
# Fitting Regressor
regressorn.fit(xn_train, yn_train)

# Making a prediction
yf_pred = regressorf.predict(xf_test)
yn_pred = regressorn.predict(xn_test)

print(mean_squared_error(yf_test, yf_pred))
print(mean_squared_error(yn_test, yn_pred))
#Results: 
# Koff Accuracy: 1.134517642079792
# Kon Accuracy: 0.3761099157240717


1.134517642079792
0.3761099157240717


In [ ]:
# Making a prediction for lanmodulin
lanM = pd.read_csv("lanM.csv") # dataset
lanmodulin = lanM.iloc[:,:]
lanM_kon = regressorn.predict(lanmodulin)
print(math.log(lanM_kon,10))
# Result: Log10 KON  [4.93466724]

# Constant from paper:(5.3 ± 0.6) × 10–12 
# Log10 Kon = 4.93466724 ± 0.3761099157240717
# Log10 koff = -6.34105689

# lower Log10 Kon = 4.558557324 
# lower kd = 4.7 × 10–12 
# lower Log10 Koff = -6.76934481

# upper Log10 Kon = 5.310777155
# upper kd = 6.2 × 10–12 
# upper Log10 Koff = -5.89683115

# thus overall range would be:
# Log10 Kon = 4.558557324 to 5.310777155
# Log10 Koff = -6.76934481 to -5.89683115


Log10 KON  [4.93466724]
